# TO DO
Visualizations  
Preprocessing  
Complete dataset  
Split by government, media, public  
**REmoveAdditional stopwords: add_stopwords = ['rt','train','southern','east','derailment','norfolk','palestine','february','u','ohio','month','week','news','please']**

# LDA analysis

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=5, suppress=True)
import pandas as pd
# data: 20 news groups

import nltk
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer 
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /Users/ribarragi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/ribarragi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ribarragi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/ribarragi/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/ribarragi/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Data preprocessing

In [2]:
# Data preprocessing (From Elaine's)
df = pd.read_csv('../data/data_tweet.csv')

# RIG (made some changes to stopwords)
add_stopwords = set(['rt','train','southern','east','derailment','norfolk','palestine','february','u','ohio'])
# add_stopwords_2 = set(['złożonych', 'ßtrong', 'áramellátása', 'áramellátást', 'ça', 'çavuş', 'çevre', 'çıkarak',
#        'çıkardı', 'çıkma', 'çıkmış', 'écologique', 'économiques', 'édito', 'élections', 'élimination', 'élites', 'éloignés', 'énième',
#        'équipes', 'était', 'état', 'états', 'étatsunis', 'étranges', 'étrangères', 'été', 'éventuellement', 'éverything', 'éviter',
#        'évoque', 'être', 'önce', 'önünde', 'última', 'üb', 'über', 'ĥmmm', 'środka', 'środowiska', 'środowisko', 'ťçh', 'że', 'ˈapəthē', 'άλλη', 'έχει', 'αλαμπάμα', 'αλλη', 'απαντηθούν', 'απεργια8μαρτη',
#        'απόδειξη', 'αρχίσουν', 'βίντεο', 'βελοπουλος', 'βουλή', 'γεννιούνται', 'δικαιολογίες', 'δουλειά', 'δυστύχημα', 'είχε', 'εκτροχιαστηκαν', 'ενα', 'επιθέσεις', 'ερωτηματα', 'ζητήσουμε',
#        'θα', 'και', 'καιρο', 'κανακη', 'κι', 'μέσω', 'μεσα', 'μηνα', 'μια', 'να', 'νδ_τελος', 'οι', 'οχαιο', 'πολλά', 'ποσα', 'ποσες', 'σήμερα', 'σε', 'στη', 'στην', 'στο', 'στράτος', 'συμπτωση', 'σχέση', 
#        'τα', 'τελευταιο', 'τεμπη', 'τεμπη_δολοφονια', 'τι', 'τις', 'το', 'τον', 'τρενα', 'τρενο', 'τρολ', 'τώρα', 'χωρες', 'аварией', 'алабаме', 'амери', 'американской', 'американском', 'анализ', 
#        'безопасность', 'величезний', 'генеральный', 'грузовой', 'давно', 'директор', 'для', 'до', 'ещё', 'жд', 'железнодорожный', 'за', 'иald', 'испытания', 'как', 'какой', 'каролина', 
#        'компании', 'конгрессом', 'лексингтоне', 'марта', 'на', 'населения', 'несколько', 'нет', 'норрис', 'огайо', 'один', 'они', 'опасностями', 'перед', 'по', 'поезд', 'показывает', 'провел',
#        'пути', 'расходы', 'рельсов', 'ремонтировали', 'северная', 'ситуации', 'сократила', 'сообщений', 'сошел', 'сошёл', 'стабильность', 'субботу', 'счёту', 'там', 'того', 'транспортной',
#        'чак', 'часов', 'что', 'штат', 'штате', 'яepublican', 'אלבמה', 'הפעם', 'أوهايو', 'از', 'اسپرینگفیلد', 'الأميركية', 'البضائع',
#        'القضبان', 'انبعاث', 'اوهایو', 'این', 'بأوهايو', 'بار', 'بحادث', 'بعد', 'تنحرف', 'ثان', 'جنوبی', 'حديدية', 'خارج', 'خروج', 'خلال', 'در', 'دیگر', 'ریل', 'سامة', 'سكة', 'شد', 'شهر', 'عربة', 'عن',
#        'في', 'قطار', 'قطارهای', 'لثاني', 'مرة', 'مسارها', 'مماثل', 'من', 'مواد', 'نورفولك', 'نورفولک', 'واحد', 'ولاية', 'یکی', 'กงานใหญ', 'กส', 'กา', 'การตกรางคร', 'การท', 'การสอบสวนเบ', 'กแล', 'ของบร',
#        'ควรจะเก', 'งของ', 'งคน', 'งท', 'งน', 'งม', 'งเล', 'งแต', 'ชาวบ', 'ฐอเมร', 'ดข', 'ดำเน', 'ตกรางเป', 'ตกลงมาเป', 'ตว', 'ทม', 'ทำให', 'ทแท', 'นการทำลายสารเคม', 'นการในภาคตะว', 'นค', 'นคร', 'นง',
#        'นบรรยากาศ', 'นพบว', 'นรถไฟบรรท', 'นอกจากน', 'นออกของสหร', 'นเด',
#        'นเพราะ', 'นไปอย', 'บต', 'บนช', 'บร', 'ผลเส', 'ฝนกรด', 'ฝนท',
#        'พบว', 'พอเค', 'ยท', 'ลอยข', 'วน', 'สารเคม', 'สำน', 'หากฝนตก',
#        'อก', 'องต', 'อน', 'อนผ', 'อยหมาของเค', 'อาจจะทำให', 'อเป', 'อได',
#        'าตายแล', 'าน', 'านคนน', 'านต', 'านไป', 'าประเภท', 'าปล', 'ายและถ',
#        'าว', 'าหมาของเค', 'าออกมาด', 'าออกมาเด', 'เก', 'เค', 'เด', 'เป',
#        'เพ', 'เพราะม', 'เล', 'เหล', 'แต', 'และไม', 'ในแอตแลนตา',
#        'ℂℍ𝔼ℝℕ𝕆𝔹𝕐𝕃', 'うひーメモ', 'か月分の食料', 'が謝罪し', 'が議会で証言する中',
#        'が議会に東パレスチナについて', 'が重要なセクターにとっては他人事では無いんだよね', 'この1カ月で2件目',
#        'この会社は民間業者として', 'この部分', 'その米国の物流ですが___________', 'ちょっと脱線し過ぎだろ',
#        'と告げるわずか数時間前に発生しました', 'と診断された', 'どうしたアメリカ', 'に対する刑事告訴がありそうだと報じている',
#        'はっきりさせるために', 'またしても', 'またまた', 'またアメリカオハイオで化学品を載せた列車の脱線事故',
#        'またオハイオで貨物列車が脱線', 'アメリカ三大ネットワークのテレビニュースを数時間遅れで視聴できる時代となっている',
#        'アラバマ州カルフーン郡でノーフォークサザンのが脱線した', 'アラバマ州速報', 'イースト',
#        'イーストパレスチナの列車脱線事故の金銭的責任を追及することを求める', 'ウェストバージニア州',
#        'ウェストバージニア州で岩石滑りに衝突した後', 'オハイオ', 'オハイオの脱線事故', 'オハイオチェルノブイリ',
#        'オハイオ州がノーフォーク', 'オハイオ州で2回目の脱線事故', 'オハイオ州での同社の列車の脱線事故は',
#        'オハイオ州での有毒化学物質流出の元トランプ当局者を非難', 'オハイオ州でノーフォーク', 'オハイオ州で別の列車が脱線',
#        'オハイオ州に救急隊員養成のための新施設を建設', 'オハイオ州のベッドフォード爆発に向かったが', 'オハイオ州の化学物質流出後',
#        'オハイオ州の司法長官がノーフォーク', 'オハイオ州は', 'オハイオ州イースト', 'オハイオ州クラーク郡でノーフォーク',
#        'オハイオ州クリーブランドで火曜日の早朝', 'オハイオ州スプリングフィールドで', 'オハイオ州スプリングフィールドでの',
#        'オハイオ州スプリングフィールドでノーフォーク', 'オハイオ州パレスチナ周辺の有毒地域について',
#        'オハイオ州西中央でノーフォーク', 'オハイオ州西中央部でノーフォーク', 'オバマ政権で強化された鉄道の安全対策が',
#        'オーバー', 'クラーク郡の住民1500人が停電の為', 'クリントン空港の外で飛行機事故で亡くなった乗組員は',
#        'クリーンアップ担当企業とは無縁だが', 'コロナ禍と露宇戦争で産業従事者激減してますしねぇ', 'サザン', 'サザンceo',
#        'サザンに対し', 'サザンの', 'サザンのがまたもや脱線', 'サザンのオハイオ州の列車の脱線事故で鉄道センサーが脚光を浴びる',
#        'サザンのスポークスパーソンは', 'サザンの列車20両が脱線', 'サザンの列車が脱線した', 'サザンの従業員が死亡した',
#        'サザンの貨物列車が脱線', 'サザンの貨物列車が脱線し', 'サザンの貨物列車が脱線したことを受けて',
#        'サザンの貨物列車が脱線した後', 'サザンを提訴', 'サザントレインが脱線', 'サザン社の脱線事故により',
#        'サザン社の車両が脱線した後', 'サザン社を提訴しました', 'サザン鉄道が脱線', 'サザン鉄道が脱線し',
#        'サザン鉄道の貨物列車が再び脱線', 'スプリングフィールドでノーフォーク', 'ダンプカーが衝突し', 'テロか',
#        'ディーゼルが川に流出', 'トランプ政権下でロビーイングで弱体化された件は注目で', 'トレイルの脱線は',
#        'ノースカロライナ州のノーフォーク', 'ノーフォーク', 'ハエのように落ちる', 'バージニア州',
#        'パレスチナの鉄道脱線事故現場からの汚染された廃棄物のすべての出荷を中止し', 'パレスチナ列車の脱線を訴える',
#        'ブラックロック', 'ブラックロックは何を考えているのだろうか', 'ブルームバーグによると',
#        'ペンシルバニア州知事がオハイオ州南部のノーフォーク列車の墜落事故を非難し', 'ホワイトハウスは共和党員',
#        'リーディング用素材', 'レールが', 'ヴァンガード', '一周后会有人帮我开奖', '不为别人想想', '世界緊急放送',
#        '东巴勒斯坦', '东巴勒斯坦的积水无需担心https', '中國人', '乗務員は流出した痕跡を見つけられませんでした',
#        '也没有化学物质泄漏', '事件沒人受傷', '事故导致1500名居民停电', '交通部長', '人が負傷', '人家天氣那麼好',
#        '今回はアラバマ州で別のノーフォーク', '今天在網上分享一個', '今年だけで脱線事故は10件以上',
#        '今度はアラバマ州でも脱線事故って', '今後米国政府も鉄道会社に対応迫ることになりそう', '今日該州', '他一直與',
#        '似乎每一天都是如此', '但我们被告知没有危险物质', '但有當地民眾表示', '住民に避難するよう指示',
#        '住民は屋内退避を命じられました', '來自', '俄亥俄', '俄亥俄州', '俄亥俄州列車脫軌事故參議院聽證會直播',
#        '俄亥俄州化學品洩漏後有另一列諾福克南方鐵路公司在北卡羅萊納州剋星敦脫軌', '俄亥俄州又有一辆norfolk', '俄亥俄州州長',
#        '俄亥俄州火车事故背后的诺福克南方公司和其他铁路公司花费数百万美元进行营销和游说', '俄亥俄州诺福克南部货运列车脱轨后',
#        '俄合俄州再有火車出軌', '假的', '先月にはイーストパレスチンでより大規模な脱線事故が発生した',
#        '先月イーストパレスチナの町で有毒化学物質が流出した脱線事故について', '先月オハイオ州で発生したノーフォーク',
#        '先月頭と今月頭にオハイオ州で脱線事故起こしたのに続き', '先週土曜日にノーフォークサザンの列車がオハイオ州で脱線し',
#        '克拉克縣', '全ての新しい住宅ローンやローンなどから利益を得るでしょう', '公衆への危険は報告されていない',
#        '公衆への危険は報告されていないと述べた', '其後再發生金屬廠爆炸事件', '再發生火車出軌事件', '出於高度謹慎的原則',
#        '出軌', '出軌地點', '出軌發生在41號州際公路附近', '出轨是常态', '刑事告発を行う', '列車と',
#        '列車の脱線に関する上院公聴会で変更を約束', '列車脱線事故', '列車脱線事故に関する上院公聴会で変更を約束',
#        '別のノーフォーク', '別のノーフォークサザン列車が脱線', '別の列車が脱線', '前17分钟是恩怨', '化学性気管支炎',
#        '卐army', '卐s', '南诺福克有什么问题吗', '危険物は積んでいないと当局が発表', '危険物処理班が現場に急行',
#        '又一起', '另一列诺福克南方货运列车出轨', '另一列诺福克南部货运列车出轨了', '可憐的',
#        '同じ鉄道会社の貨物列車がまた脱線事故', '同時に複数地区で停電も起きているらしい', '呼籲市民就地避難',
#        '因为我们听说俄亥俄州又有一列火车出轨了', '因为福禄寿利用我赚了千万还恶心我', '土曜日', '在內的聯邦官員保持聯繫',
#        '在週六諾福克南方springfield火車出軌後', '堆疊在路軌旁', '報告說', '報復攻撃', '墮軌',
#        '大手製薬会社との繋がりは', '大規模な', '大量逮捕', '太缺德了', '妙なモノは漏れたりしてないそうだけど', '安全',
#        '导致北卡罗来纳州延误', '少なくとも', '就在几天前', '居民被告知就地避难', '屋内退避命令',
#        '州内では有害物質が流出した先月の事故に続き２例目', '布蒂吉格说美国一年有一千', '幼児教育など', '当地老百姓怕啥呢',
#        '当局によると', '当該企業が欠陥テストを実施していることは承知', '很可能会被报复', '必然的に混ざり合っていました',
#        '必需品を手元に用意しておくことをお勧めします', '怎麼會有人病倒', '怠慢か', '情報弱者さんへ', '意外',
#        '我忍无可忍选择曝光福禄寿利用警察冻卡', '戦争か', '投稿時間', '拉文纳', '拜登', '拡散希望',
#        '據報這列載貨火車共有212卡車廂', '收钱解冻', '政府督促居民就地寻求庇护', '政治家もわかっているか怪しい', '斯托本维尔市', '斯普林菲尔德', '日ミシシッピ川にフッ化水素酸が流出', '日本のメディアは理解していない',
#        '显然正在进行', '時代で立ち往生していることを示しています', '暂时没有人员伤亡', '朝起きて目に飛び込んできたニュース', '本來是逗大家一樂的', '来自', '東パ',
#        '東パレスチナに鉄道業界のロビイストにそれらを売り渡したことに対する謝罪を負っています', '東パレスチナの住民と労働者が',
#        '東パレスチナの流出後', '桑德斯基', '汚染の規模が大きくて話題になっているけど', '沒想到第一條留言便是簡短中文字的',
#        '波蒂奇县', '洩漏化學物污染當地', '海外tech', '海外科学', '深くお詫び申し上げます', '火曜日',
#        '狗子在林中狂奔的視頻', '現場に調査員を派遣', '現金', '環境保護庁はオハイオ州の有毒列車事故による汚染廃棄物の出荷の一時停止を命令', '環境保護庁はノーフォーク', '用心深さから',
#        '當中20節出軌', '當局稱車上並無載有有毒化學物', '疯狂三月', '皮特', '皮特布蒂吉格', '看到有毒化學物品處理組的車輛出現', '等であり', '米では年平均1700件の脱線事故が発生', '米国運輸安全委員会は',
#        '米版', '緊急事務管理局', '繼上月發生火車出軌事件', '羊民们别瞎担心了', '美国', '美国俄亥俄州再发生列车脱轨事故', '美国俄亥俄州载有毒化学品列车脱轨',
#        '美国环保局阻止诺福克南方公司清除有毒的俄亥俄州火车脱轨残骸', '肇事的火車同樣屬於norfolk', '脱線させるひとをディレーラーと呼ぶ', '脱線はderail', '脱線ばっかりしてるな', '脱軌', '脱轨',
#        '自分たちが引き起こした混乱の後始末の代償を払うことになる', '西行小宝', '調査員を派遣している', '諾福克南方公司', '證實', '議会の共和党員と元トランプ政権の当局者は', '诺福克南方声称居民没什么好担心的', '貨物列車が脱線',
#        '軽油が川に流出', '转发评论此条推特抽一万u', '还有u商安徽省芜湖防空洞帮人藏匿的数十亿资产', '这可能是我的最后一条视频', '这次是在北卡罗来纳州', '这次是在斯普林菲尔德', '速報', '週末には',
#        '適切な処分を確実にするよう命じた', '避難するよう求められていました', '避難指示も', '邁克德溫', '都散了吧', '鄉下', '鉄道の', '鉄道事業者のノーフォーク',
#        '鉄道会社が事故防止を目的として使用しているセンサーの役割に改めて注目が集まっています', '鐵路', '铁路', '阿尼斯顿', '阿拉巴馬現場脫軌為聽證會助興', '阿拉巴马州', '附近居民被要求就地避难',
#        '陰謀集團的貝萊德公司就是俄亥俄州火車車禍的主要股東之一', '高速鉄道路線の筆頭株主は中央銀行家', '黑岩貝萊德擁有之一股東',
#        '거주하는', '공무원들이', '근처에', '내렸다', '노퍽', '대피소로', '로이터', '만에', '명령을', '바이든은', '발생해', '방문하지', '번도', '번째', '번째로', '사고', '사고가', '서던', '않은',
#        '연루된', '열차', '열차가', '오하이오', '오하이오에서', '오하이오주에서', '주민들에게', '지역', '찾은', '철도가', '출처', '탈선', '탈선해', '토요일', '트럼프', '현장', '현지'])


remove_stopwords = set(["no", "not", "nor", "against", "aren't", "couldn't", "didn't", 
                                                   "doesn't", "don't", "haven't", "hadn't", "hasn't", "isn't", "mightn't",
                                                   "mustn't", "needn't", "shouldn't", "wasn't", "weren't", "wouldn't"])
# remove stopwords (not impacting sentiment analysis) and punctuations
stop_words = set(stopwords.words('english')) - remove_stopwords
stop_words = set(stopwords.words('english')).union(add_stopwords)
# stop_words = set(stopwords.words('english')).union(add_stopwords_2)
tokenizer = RegexpTokenizer(r'\w+')


# normalize pos tags
lemmatizer = WordNetLemmatizer()

def pos_tagger(nltk_tag):
  if nltk_tag.startswith('J'):
      return wordnet.ADJ
  elif nltk_tag.startswith('V'):
      return wordnet.VERB
  elif nltk_tag.startswith('N'):
      return wordnet.NOUN
  elif nltk_tag.startswith('R'):
      return wordnet.ADV
  else:         
      return None


# lemmatize each token with pos tag
def lemma_token(row):
  wordnet_tagged = list(map(lambda x: (x[0], pos_tagger(x[1])), row))
  lemmatized = []
  for word, tag in wordnet_tagged:
    if tag is None:
        # if there is no available tag, just append the original token
        lemmatized.append(word)
    else:       
        # else use the pos tag to lemmatize the token
        lemmatized.append(lemmatizer.lemmatize(word, tag))
  return lemmatized


# RIG (edit: added http://)
# remove urls
df['Message_no_url'] = df['MESSAGE'].apply(lambda x: " ".join([word for word in x.split(" ") if not(word.startswith(("https://", 'http://')))]))


# tokenize MESSAGE and remove stopwords
df['Text'] = df['Message_no_url'].map(tokenizer.tokenize)
df['Text'] = df['Text'].apply(lambda x : nltk.pos_tag([item.lower() for item in x if item.lower() not in stop_words]))

# apply lemmatization and reset index
df['Text'] = df['Text'].apply(lambda row : lemma_token(row))
df = df.reset_index(drop=True)

# join words
df['T_text'] = df['Text'].apply(lambda row : " ".join(row))

In [3]:
stop_words

{'a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'derailment',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'east',
 'february',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 'her',
 'here',
 'hers',
 'herself',
 'him',
 'himself',
 'his',
 'how',
 'i',
 'if',
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it's",
 'its',
 'itself',
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'norfolk',
 'not',
 'now',
 'o',
 'of',
 'off',
 'ohio',
 'on',
 'once',
 'only',
 'or',
 'other',
 

In [4]:
len(df)

85220

In [5]:
# Not rndom sample
sample_data = df.T_text.tolist()
del df

In [6]:
# # ## We first used a randome sample to test this
# import random
# sample_size = 10000
# random.seed(1)
# # We will use a random sample of 10k tweets to test
# idx_sample = random.sample(range(len(df.T_text)), sample_size)
# # sample_data is a list of strings, each string is a tweet. ssample_Data is length 10k
# sample_data = [df.T_text[i] for i in idx_sample]

In [7]:
len(sample_data)

85220

We use CountVectorizer model to get term frequencies.

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
# We could cap the size of the vocab to vocab_size
# If not capped, we have 48k words
max_vocabulary = 10000

# We can also set options to only keep words that appear in at least this percentage of documents
# - min_df = 2 # words in less than 2 documents we discard them
# And we can choose to only keep words that appear in at most this percentage of documents:
# - max_df = 0.95 # words that appear in more than 95% of documents, we discard them
# Finally, we can set max_features, to keep only the most frequently occuring words
# - max_features = max_vocabulary # after I do the max_df, min_df and stopwords filetrs, I look at how many words I have left: if its greater than
# max_features, I only keep the most popular. BUT, if the words left after the 3 filters is less than max_features, then it will be ignored. 


# tf_vectorizer = CountVectorizer(max_df=0.95,
#                                 min_df=2,
#                                 stop_words='english',
#                                 max_features=max_vocabulary)

# tf_vectorizer = CountVectorizer(stop_words='english')

tf_vectorizer = CountVectorizer(min_df = 2)

# tf: term frequency vectorizer

# the fitting is learning the vocabulary: goes thru all the data and know the words it needs to keep track of
# transform: Ive done the fitting, Ill go back to data points and convert each one into a feat vector representation using the vocab i learnt during fitting
tf_fit = tf_vectorizer.fit(sample_data)
tf = tf_vectorizer.fit_transform(sample_data)


In [9]:
# size of the matrix (documents, words)
tf.toarray().shape

(85220, 23665)

In [10]:
# seems like the highest term-doc freq is 10
max_freq = tf.toarray().max()
max_freq

10

In [11]:
# Just for curiosity, I want to know which doc / word combination has highest valeus
max_freq_idx = list(np.argwhere(tf.toarray()==max_freq)[0])
max_freq_doc, max_freq_word = max_freq_idx
# to print out the vocabulary in the doc-word freq matrix
tf_vectorizer.get_feature_names_out()[max_freq_word]
# In which document is this word?
sample_data[max_freq_doc]

'trump state go trump trump trump trump trump trump trump trump trump'

In [12]:
# we can get the whole vocabulary, sorted alphabetically
tf_vectorizer.get_feature_names_out()[-1000:]

array(['wearing', 'weartv', 'weary', 'weasel', 'weaselzippers', 'weather',
       'weatherchannel', 'weatherfox64', 'weaver', 'web', 'web3',
       'webcast', 'weber', 'webershandwick', 'webguytv', 'webpage',
       'website', 'websterscat', 'wed', 'wedge', 'wednesday',
       'wednesdaymorning', 'wednesdaymotivation', 'wednesdaythought',
       'wednesdaywisdom', 'wee', 'weed', 'week', 'weekday', 'weekend',
       'weeklong', 'weekly', 'weeklywrapup', 'weep', 'weer', 'wef',
       'weflysoon', 'wehearpodcast', 'weigh', 'weighs', 'weight', 'weir',
       'weird', 'weirdly', 'weirdo', 'weirdos', 'weirton', 'weisman',
       'welcome', 'welfare', 'well', 'wellbeing', 'wellknownequine',
       'wellness', 'wellsfargo', 'wellsh40', 'welovepresidenttrump',
       'welp', 'welt', 'wemyss_b', 'wendoverpro', 'wendy',
       'wendybugliari', 'wendyldahl', 'wendynas', 'wendyp4545',
       'wendywi89642313', 'wepeoplefreedom', 'weptsmile', 'werent',
       'werewolf_doctor', 'wes', 'wesa', 'wesh'

We can also go in reverse: given a word, we can figure out which column index it corresponds to. To do this, we use the `vocabulary_` attribute.

In [13]:
tf_vectorizer.vocabulary_['car']
# .vocabulary_ get the word index in the vocabulary
# IN OTHER WORDS: what column do I have to look at to find the word car

3596

We can figure out what the raw counts are for the 0-th post as follows.

In [14]:
tf.toarray().shape

(85220, 23665)

In [15]:
tf[0].toarray()

# I can index in to the zeroeth row of the matrix and convert into an array, I am anazlizing just the zeroeth row:
# feature vector representation for the zeroeth document: 
# We should be able to write code to know what word corresponds each non zero number there

# tf_vectorizer.get_feature_names_out()[(tf[0].toarray() != 0)[0]] # LIKE THIS!

# run a for loop that goes thru the entries, and check to which indeces this corresponds in the vocabulary

array([[0, 0, 0, ..., 0, 0, 0]])

The fitting procedure determines the every topic's distribution over words; this information is stored in the `components_` attribute. There's a catch: we actually have to normalize to get the probability distributions (without this normalization, instead what the model has are pseudocounts for how often different words appear per topic).

We now fit an LDA model to the data.

In [16]:
# Now LDA. This will take a while ong

num_topics = 5

from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=num_topics, random_state=0)
lda.fit(tf)

LatentDirichletAllocation(n_components=5, random_state=0)

In [17]:
lda.components_.shape
# The shaope of this is 16 by 1000, 10 topics and 1000 words
# These are not probability distributions yet

(5, 23665)

In [18]:
# If we access: sum ACROSS COLUMNS (get rid of columns), I get 6 numbers: a sum for each row.
# They dont sum uop to 1, because not probability distributions

# .components_ is a raw count histogram, but why are there fractions?
# Because of how it works: when it estimates how much a word belongs to a topic it does a prob assignment, not a deterministic assignment.
lda.components_.sum(axis=1)

array([212806.09756, 183229.36112, 226403.02355, 249360.46048,
       196956.05727])

In [19]:
topic_word_distributions = np.array([row / row.sum() for row in lda.components_])
# We can get the proba distributions: take each row and divide by sum, then we get what was in the purple box in the lecture slide:
# 

In [20]:
topic_word_distributions.shape

(5, 23665)

We can verify that each topic's word distribution sums to 1.

In [21]:
topic_word_distributions.sum(axis=1)
# then we have distributions: every row sums to 1

array([1., 1., 1., 1., 1.])

We can also print out what the probabilities for the different words are for a specific topic. This isn't very easy to interpret.

In [22]:
print(topic_word_distributions[4])
# lets look at 0th topic, it is saying that the 0th word appears 0.00011, word 1 appears with prob 0.00191
# Lets view the data differently to interpret

[0.      0.      0.      ... 0.00001 0.00001 0.00001]


Instead, usually people do something like looking at the most probable words per topic, and try to use these words to interpret what the different topics correspond to.

In [23]:
# we can sort the probability by big to small
# for each topic, listing top 20 most popular words (by probability)

num_top_words = 20

def print_top_words(topic_word_distributions, num_top_words, vectorizer):
    vocab = vectorizer.get_feature_names_out()
    num_topics = len(topic_word_distributions)
    print('Displaying the top %d words per topic and their probabilities within the topic...' % num_top_words)
    print()

    for topic_idx in range(num_topics):
        print('[Topic ', topic_idx, ']', sep='')
        # This is the interesting part: take the topic_word_distributions only for the topic_idxth topic, and get the indexes sorted from highest to lowest
        sort_indices = np.argsort(-topic_word_distributions[topic_idx])
        # then for the range 0,num_words, print the vocab of each of the words and its entry in the topic_word_distribution matrix
        for rank in range(num_top_words):
            word_idx = sort_indices[rank]
            print(vocab[word_idx], ':',
                  topic_word_distributions[topic_idx, word_idx])
        print()

print_top_words(topic_word_distributions, num_top_words, tf_vectorizer)

Displaying the top 20 words per topic and their probabilities within the topic...

[Topic 0]
toxic : 0.022457791846392967
ceo : 0.01909519770061295
resident : 0.017347292501322095
chemical : 0.016160113157076755
epa : 0.013899063024322483
via : 0.009893960751737617
pay : 0.008269276069613581
say : 0.008163893554262587
senate : 0.008134594278846838
tell : 0.007605117286419225
congress : 0.006781515182579619
order : 0.006306676179334696
clean : 0.006226100852638858
rail : 0.0062106600464542985
crash : 0.006076923915123679
site : 0.005999986680991488
worker : 0.0059216764863732185
animal : 0.005689835953484064
company : 0.00549751936463235
demand : 0.005352323604188377

[Topic 1]
derail : 0.041301288720680895
another : 0.03180513917666337
car : 0.026418350367137678
derails : 0.022203092403190045
hazardous : 0.019883504315243262
material : 0.015784198361090688
say : 0.013526961781535762
springfield : 0.012509990870489616
20 : 0.011057471751499764
company : 0.010800731026674894
alabama : 0.

### To do
Present these results and then tweak:
- Present this
- Divid by govt people and public and present results
- set a minimum frequency for the terms included and a max for vocabulary
- Present overall topics
- Divide the set by govt, media an public and redo analysis

We can use the `transform()` function to figure out for each document, what fraction of it is explained by each of the topics.

In [ ]:
# consider removing:
terms_remove = ['train', 'https', 'rt', 'palestine', 'east', 'ohio', 'norfolk', 'southern']

In [ ]:
doc_topic_matrix = lda.transform(tf)
# Ill get a low dimensional version of the data, the document topic matrix (10, 10)
# each of the 10,000 represented as a distribution over 10 different topics

In [ ]:
doc_topic_matrix.shape

In [ ]:
# The topic distribution of the 0th document is very small except for 0.96288 fior index 7
# We saw from above, topic 7 is about religion, and we can check that document 0 is about religion
doc_topic_matrix[0]


# Note: for GMM when you do the .predict, itll give you k probabilites that sum to 1. 
# predict_proba of GMM 
# here: ... gives you a proba distr:
# INTERPRETARATION DIFFERENT

# FOR LDA: document can consist of a bunch of words, and different fraction of the words are truly in different topics: we get mixed membership, words are allowed to e in different topics
# GMM: the above doesnt hold

In [ ]:
doc_topic_matrix[0].sum()

Note that this *could* be interpreted as a form of dimensionality reduction: document 0 is converted from its raw counts histogram representation to a 10-dimensional vector of probabilities, indicating estimated memberships to the 10 different topics.

# **--------------------- Stop here by now ---------------------**

## Word clouds

Here's a fancier way to visualize. This requires installation of the wordcloud package:

```
pip install wordcloud
```

In [ ]:
from wordcloud import WordCloud

num_max_word_cloud_words = 100

vocab = tf_vectorizer.get_feature_names()
num_topics = len(topic_word_distributions)

for topic_idx in range(num_topics):
    wc = WordCloud(max_words=num_max_word_cloud_words)
    wc.generate_from_frequencies(dict(zip(vocab, topic_word_distributions[topic_idx])))
    plt.figure()
    plt.imshow(wc, interpolation='bilinear')
    plt.title('Topic %d' % topic_idx)

## Computing co-occurrences of words

Here, we count the number of newsgroup posts in which two words both occur. This part of the demo should feel like a review of co-occurrence analysis from earlier in the course, except now we use scikit-learn's built-in CountVectorizer. Conceptually everything else in the same as before.

In [ ]:
word1 = 'year'
word2 = 'team'

word1_column_idx = tf_vectorizer.vocabulary_[word1]
word2_column_idx = tf_vectorizer.vocabulary_[word2]

In [ ]:
np.array(tf.todense())

In [ ]:
tf[:, word1_column_idx].toarray()

In [ ]:
documents_with_word1 = (tf[:, word1_column_idx].toarray().flatten() > 0)

In [ ]:
documents_with_word2 = (tf[:, word2_column_idx].toarray().flatten() > 0)

In [ ]:
documents_with_both_word1_and_word2 = documents_with_word1 * documents_with_word2

In [ ]:
documents_with_both_word1_and_word2

Next, we compute the log of the conditional probability of word 1 appearing given that word 2 appeared, where we add in a little bit of a fudge factor in the numerator (in this case, it's actually not needed but some times you do have two words that do not co-occur for which you run into a numerical issue due to taking the log of 0).

In [ ]:
eps = 0.1
np.log2((documents_with_both_word1_and_word2.sum() + eps) / documents_with_word2.sum())

In [ ]:
# compute log of prob of see one word given you see another, using count vectorizer!
# Same content as in hw , but with a different tool
# This uses np.arrays instead of counters. Needs to kep track of the indexes.
# Understad how countvectorizer works

def prob_see_word1_given_see_word2(word1, word2, vectorizer, eps=0.1):
    word1_column_idx = vectorizer.vocabulary_[word1]
    word2_column_idx = vectorizer.vocabulary_[word2]
    documents_with_word1 = (tf[:, word1_column_idx].toarray().flatten() > 0)
    documents_with_word2 = (tf[:, word2_column_idx].toarray().flatten() > 0)
    documents_with_both_word1_and_word2 = documents_with_word1 * documents_with_word2
    return np.log2((documents_with_both_word1_and_word2.sum() + eps) / documents_with_word2.sum())

In [ ]:
type(word1), type(word2)

## Topic coherence

The below code shows how one implements the topic coherence calculation from lecture.

In [ ]:
# In the cell below, vectorizer.get_feature_names() is just tokenizing the text, 
# eliminating stopwords, eliminating some other words, and then giving you back
# a list of the tokens in string format

In [ ]:
# topic coherence

def compute_average_coherence(topic_word_distributions, num_top_words, vectorizer, verbose=True):
    vocab = vectorizer.get_feature_names()
    num_topics = len(topic_word_distributions)
    average_coherence = 0
    # foor loop thru different topics, for each topic double nested for loop, going thru each entry, ordering matters
    # make sure words arent the same, then compute the log(prob)
    # add a bunhc of them and divide by total number of topics
    for topic_idx in range(num_topics):
        if verbose:
            print('[Topic ', topic_idx, ']', sep='')
        
        sort_indices = np.argsort(topic_word_distributions[topic_idx])[::-1]
        coherence = 0.
        for top_word_idx1 in sort_indices[:num_top_words]:
            word1 = vocab[top_word_idx1]
            for top_word_idx2 in sort_indices[:num_top_words]:
                word2 = vocab[top_word_idx2]
                if top_word_idx1 != top_word_idx2:
                    coherence += prob_see_word1_given_see_word2(word1, word2, vectorizer, 0.1)
        
        if verbose:
            print('Coherence:', coherence)
            print()
        average_coherence += coherence
    average_coherence /= num_topics
    if verbose:
        print('Average coherence:', average_coherence)
    return average_coherence

In [ ]:
compute_average_coherence(topic_word_distributions, num_top_words, tf_vectorizer, True)
# These are negative, the highest possible is zero.

## Number of unique words

The below code shows how one implements the number of unique words calculation from lecture.

In [ ]:
# brute force implementation
# nothing clever to speed upmcalculation

#fro loop for each topic
# For loop over each top word
# another for loop for other topics
# lok for all other top words in those other topics
# check for uniqueness

def compute_average_num_unique_words(topic_word_distributions, num_top_words, vectorizer, verbose=True):
    vocab = vectorizer.get_feature_names()
    num_topics = len(topic_word_distributions)
    average_number_of_unique_top_words = 0
    for topic_idx1 in range(num_topics):
        if verbose:
            print('[Topic ', topic_idx1, ']', sep='')
        
        sort_indices1 = np.argsort(topic_word_distributions[topic_idx1])[::-1]
        num_unique_top_words = 0
        for top_word_idx1 in sort_indices1[:num_top_words]:
            word1 = vocab[top_word_idx1]
            break_ = False
            for topic_idx2 in range(num_topics):
                if topic_idx1 != topic_idx2:
                    sort_indices2 = np.argsort(topic_word_distributions[topic_idx2])[::-1]
                    for top_word_idx2 in sort_indices2[:num_top_words]:
                        word2 = vocab[top_word_idx2]
                        if word1 == word2:
                            break_ = True
                            break
                    if break_:
                        break
            else:
                num_unique_top_words += 1
        if verbose:
            print('Number of unique top words:', num_unique_top_words)
            print()

        average_number_of_unique_top_words += num_unique_top_words
    average_number_of_unique_top_words /= num_topics
    
    if verbose:
        print('Average number of unique top words:', average_number_of_unique_top_words)
    
    return average_number_of_unique_top_words

In [ ]:
compute_average_num_unique_words(topic_word_distributions, num_top_words, tf_vectorizer, True)

## Plotting average coherence vs k (number of topics), and average number of unique words vs k

Next, we plot the average coherence vs k and the average number of unique words vs k. Note that these are *not* the only topic model metrics available (much like how CH index is not the only metric available for clustering).

For both average coherence and average number of unique words, we would like these to be high. In this particular example, it turns out k=2 yields very high values for both but if you look at the topics learned for k=2, they are qualitatively quite bad (basically one topic is gibberish and the other is everything else!). This observation reinforces the important idea that while there exist topic modeling metrics (such as coherence and number of unique words), you should definitely still look at what the learned topics are (e.g., by printing the top words per topic) to help decide on what value of k to use.

Also, keep in mind that the results are in some sense "noisy" since the LDA fitting procedure is random. We're choosing a specific `random_state` seed value but if we try different random seeds, we can get different results. For simplicity, because LDA fitting is quite computationally expensive, we are *not* doing what we did with GMM's where we did many different random initializations. Thus, the conclusions we draw regarding how many topics to use might actually be different with different random initializations.

At least according to average coherence and average number of unique words for the random seed we use, the results below suggests that using k=4 yields average coherence and average number of unique words that are still reasonably high (as good as or almost as good as the k=2 result), and inspecting the topics learned for k=4, they are definitely more interesting than the ones learned for k=2.

From qualitatively looking at topics, the k=5, k=6, and k=7 topics also look decent. When k gets too large (e.g., k=10), there start to be topics that look like there might be too much overlap (such as multiple topics that seem to be about computers).

Note that one of the things to look out for is whether there are "stable" topics, where even for slightly different values of k and different random initializations, LDA keeps finding specific topics (e.g., one on gibberish, one on numbers, etc).

In [ ]:
k_values = range(2, 11)
avg_coherences = []
avg_num_unique_words = []

for k in k_values:
    lda_candidate = LatentDirichletAllocation(n_components=k, random_state=0)
    lda_candidate.fit(tf)
    topic_word_distributions = np.array([row / row.sum() for row in lda_candidate.components_])
    print('-' * 80)
    print('Number of topics:', k)
    print()
    print_top_words(topic_word_distributions, num_top_words, tf_vectorizer)
    print()
    print()
    
    avg_coherences.append(compute_average_coherence(topic_word_distributions, num_top_words, tf_vectorizer, False))
    avg_num_unique_words.append(compute_average_num_unique_words(topic_word_distributions, num_top_words, tf_vectorizer, False))

In [ ]:
plt.plot(k_values, avg_coherences)
plt.xlabel('Number of topics')
plt.ylabel('Average coherence')

# For differnt number of topics, I fitted a different LDA



In [ ]:
plt.plot(k_values, avg_num_unique_words)
plt.xlabel('Number of topics')
plt.ylabel('Average number of unique words')

# For different topic models, the average number of unique words
# We want coherence to be high and unique words to be high

# Number of topic = 2, gets me high coherence and hogh unique words. BUT, for model 2, there are 2 topics: garbage and not garbage!
# that doesnt help a lot

# So, you want to tolerate a lower value for one of the metrics, but look at the actual topics to interpret what is going on
# If we see resutlts of 4, starts to make more ssense but feels like still low
# go to 5, starts to make a bit more sense

